### Setup domain:

notebooks/Experimental/Ishan/AA/setup-data.ipynb

In [1]:
import syft as sy

In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
domain.datasets

Idx,Name,Description,Assets,Id
[0],BreastCancerDataset,Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. The modified dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. Patches of size 50 x 50 were extracted from the original image. The labels 0 is non-IDC and 1 is IDC.,"[""train_images""] -> [""train_labels""] ->",f9d7fc06-3262-4d0a-8fb0-4460d51fafbc


In [4]:
X_train = domain.datasets[-1]["train_images"][:2]
y_train = domain.datasets[-1]["train_labels"][:2]

In [5]:
input_shape = X_train[:2].public_shape

In [6]:
input_shape

(2, 3, 50, 50)

In [7]:
from syft import nn

In [8]:
model = nn.Model()

# Layer 1
model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=input_shape))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 2
model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 3
model.add(nn.Convolution(nb_filter=128, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 4
model.add(nn.Convolution(nb_filter=256, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 5
model.add(nn.AvgPool(5))

# Layer 6
model.add(nn.Flatten())

# Layer 7
model.add(nn.Linear(2, 512))

In [9]:
model.initialize_weights()

In [10]:
model.layers

In [11]:
model_ptr = model.send(domain,send_to_blob_storage=False)

In [12]:
x_batch, y_batch = X_train[:2], y_train[:2]

In [13]:
import time
from tqdm import tqdm
n_epochs = 1
batch_size = 2
for epoch in tqdm(range(n_epochs)):
    print(f"Epoch: {epoch}")
    total_images = X_train.public_shape[0]
    start_idx = 0
    while(start_idx < total_images):
        end_idx = start_idx+batch_size
        x_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]
        run_status = model_ptr.step(x_batch, y_batch)
        while(not run_status.exists):
            time.sleep(10)
        print("Epoch completed.")
        start_idx += batch_size

  0%|                                                                                                                           | 0/1 [00:00<?, ?it/s]

Epoch: 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [08:11<00:00, 491.47s/it]

Epoch completed.


In [14]:
# Need to implement download_weights
weights = model_ptr.download_weights(sigma=1000)

In [ ]:
while not weights.exists:
    time.sleep(5)

In [ ]:
weights.get_copy()